# Praktek kedua : Menggunakan Model Pre-trained untuk pengenalan objek

## Impor library yang diperlukan untuk membangun dan melatih model

In [1]:
import tensorflow as tf  # Library utama untuk machine learning dan deep learning, menyediakan alat untuk membangun dan melatih model.
from tensorflow import keras  # API tingkat tinggi dari TensorFlow yang mempermudah pembuatan dan pelatihan model neural network.
from tensorflow.keras import layers  # Modul untuk mendefinisikan lapisan jaringan saraf seperti konvolusi (Conv2D), fully connected (Dense), dll.
from tensorflow.keras.applications import MobileNet  # Modul untuk memuat model MobileNet yang telah dilatih sebelumnya (pre-trained).
import numpy as np  # Library untuk operasi numerik seperti manipulasi array dan perhitungan matematika.
import matplotlib.pyplot as plt  # Library untuk visualisasi data, digunakan untuk menampilkan gambar atau grafik hasil.


## Memuat dataset MNSIT

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# Memuat dataset MNIST yang berisi gambar digit tulis tangan. x_train (60.000 gambar) dan y_train (labelnya) untuk pelatihan,
# x_test (10.000 gambar) dan y_test (labelnya) untuk pengujian. Setiap gambar berukuran 28x28 piksel dalam format grayscale.

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Menormalkan data dan mengubah bentuk untuk MobileNet

In [3]:
x_train = x_train / 255.0  # Membagi nilai piksel (awalnya 0-255) dengan 255 untuk menormalkan ke rentang 0-1,
# meningkatkan stabilitas dan kecepatan konvergensi selama pelatihan.
x_test = x_test / 255.0  # Melakukan normalisasi yang sama pada data uji agar konsisten dengan data latih.

In [4]:
# Menambah dimensi kanal
x_train = x_train[..., np.newaxis]  # Menambah dimensi ke-4 (kanal) pada x_train, dari (60000, 28, 28) menjadi (60000, 28, 28, 1).
# CNN membutuhkan format input (batch, height, width, channels), di sini 1 kanal karena gambar grayscale.
x_test = x_test[..., np.newaxis]  # Melakukan hal yang sama pada x_test, dari (10000, 28, 28) menjadi (10000, 28, 28, 1).

In [5]:
# Fungsi untuk memproses gambar
def preprocess_image(image, label):  # Mendefinisikan fungsi untuk memproses gambar sebelum dimasukkan ke model.
    image = tf.image.resize(image, [224, 224])  # Mengubah ukuran gambar dari 28x28 menjadi 224x224, sesuai kebutuhan input MobileNet.
    image = tf.image.grayscale_to_rgb(image)    # Mengonversi gambar grayscale (1 kanal) menjadi RGB (3 kanal) dengan mengulang nilai intensitas.
    return image, label  # Mengembalikan gambar yang telah diproses beserta labelnya.

In [6]:
# Membuat dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))  # Membuat objek dataset dari pasangan data latih (x_train) dan label (y_train).
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))  # Membuat objek dataset dari pasangan data uji (x_test) dan label (y_test).

In [7]:
# Memproses dan membuat batch
train_dataset = train_dataset.map(preprocess_image).batch(32)  # Menerapkan fungsi preprocess_image ke setiap elemen dataset latih,
# lalu membaginya menjadi batch berukuran 32 untuk efisiensi pelatihan.
test_dataset = test_dataset.map(preprocess_image).batch(32)  # Melakukan hal yang sama pada dataset uji untuk konsistensi.

## Memuat model MobileNet pre-trained

In [8]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Memuat model MobileNet yang telah dilatih pada dataset ImageNet. include_top=False berarti lapisan klasifikasi atas dihilangkan,
# input_shape=(224, 224, 3) sesuai dengan ukuran input yang diharapkan (RGB).
base_model.trainable = False  # Membekukan bobot MobileNet agar tidak diperbarui selama pelatihan (menggunakan transfer learning).

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Menambahkan lapisan kustom untuk klasifikasi MNSIT

In [9]:
model = keras.Sequential([  # Membuat model berurutan (sequential) dengan menumpuk lapisan-lapisan.
    base_model,  # Lapisan pertama adalah MobileNet pre-trained sebagai fitur ekstraktor.
    layers.GlobalAveragePooling2D(),  # Mengubah output MobileNet (berupa peta fitur 3D) menjadi vektor 1D dengan rata-rata global.
    layers.Dense(128, activation='relu'),  # Lapisan fully connected dengan 128 neuron dan aktivasi ReLU untuk menangkap pola non-linear.
    layers.Dense(10, activation='softmax')  # Lapisan output dengan 10 neuron (sesuai jumlah kelas 0-9) dan aktivasi softmax untuk probabilitas.
])


## Mengompilasi model

In [10]:
model.compile(optimizer='adam',  # Mengatur optimizer 'adam', algoritma penyesuaian bobot berbasis gradien yang adaptif.
              loss='sparse_categorical_crossentropy',  # Fungsi loss untuk klasifikasi multi-kelas dengan label integer (0-9).
              metrics=['accuracy'])  # Melacak akurasi sebagai metrik performa selama pelatihan dan evaluasi.

## Melatih model

In [11]:
model.fit(train_dataset, epochs=5)  # Melatih model menggunakan dataset latih selama 5 epoch (iterasi penuh).
# Setiap epoch menampilkan loss dan akurasi pada data latih.


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.9242 - loss: 0.2451
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9788 - loss: 0.0702
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9840 - loss: 0.0492
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9871 - loss: 0.0382
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.9901 - loss: 0.0294


## Mengevaluasi model

In [12]:
test_loss, test_acc = model.evaluate(test_dataset)  # Menguji model pada dataset uji, menghitung loss dan akurasi pada data yang belum dilihat.
print(f'Akurasi pada data uji: {test_acc:.4f}')  # Mencetak akurasi uji dengan presisi 4 desimal untuk evaluasi performa.


313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9784 - loss: 0.0834
Akurasi pada data uji: 0.9807
